# Part 7 Exponents and Radicals

In [ ]:
#constructor
import sympy
import random
from string import ascii_lowercase
from string import ascii_uppercase
from copy import copy

# gather up alphanumeric charectors we might want to use for variable names
alpha = [i for i in ascii_uppercase + ascii_lowercase]
# remove the ones that might be confusing in a problem
alpha.remove("l")
alpha.remove("o")
alpha.remove("O")
alpha.remove("I")
alpha.remove("i")
# gather up numerical digits we might want to use for coefficients
# nothing special about -26 to 26, other than it matches the number of chars
# above
digits = [i for i in range(-26, 26)]
# make a list of the nums above, but with zero removed. This way we know we
# can always guarantee selection of a non-zero digit (so the degree of a
# polynomial in an equation is at least a certain value)
digits_nozero = [i for i in range(-26, 26)]
digits_nozero.remove(0)


def shuffle(x):
    x = list(x)
    random.shuffle(x)
    return x


def get_coefficients(n, exclude=["x", "X"], first_nonzero=True, var_coeffs=False,
                     reduce=True):
    """
    Helper function to generate "good" coefficients for problems
    """
    if var_coeffs:
        selection = copy(digits_nozero + alpha)
        for i in exclude:
            selection.remove(i)
    else:
        selection = digits_nozero
    coeffs = []
    for i in range(n):
        c = random.choice(selection)
        if isinstance(c, str):
            c = sympy.Symbol(c)
        if reduce and random.randint(0, 1):
            c = 0
        coeffs.append(c)
    if first_nonzero and coeffs[0] == 0:
        coeffs[0] = random.choice(selection)
    return coeffs


def render(expr, lhs=""):
    """
    Puts $ at the beginning and end of a latex expression.
    lhs : if we want to render something like: $x = 3 + 5$, set the left hand 
          side here
    """
    left = "$$"
    if lhs:
        left = "$$%s =" % lhs
    return ''.join([left, sympy.latex(expr), "$$"])

In [ ]:
import sympy
from sympy.parsing.sympy_parser import parse_expr
from sympy.polys.polytools import degree
import random


def make_quadratic_eq(var="x", rhs=None, integer=[0, 1]):
    """
    Generates quadratic equation problem expression and
    set of solutions
    x : charector for the variable to be solved for. defaults to "x".
                            OR
        a list of possible charectors. A random selection will be made from them.
    rhs : value to set for the right-hand side. If not given, the 
          right-hand side will be a randomly generated polynomial expression
          of degree <= 2, in the same variable.
    integer : determines whether generated problem will have integer roots or
              not. Default is a random selection.
    """
    if isinstance(var, str):
        var = sympy.Symbol(var)
    elif isinstance(var, list):
        var = sympy.Symbol(random.choice(var))
    if isinstance(integer, list):
        integer = random.choice(integer)
    if integer:
        r1 = random.choice(digits_nozero)
        r2 = random.choice(digits_nozero)
        lhs = (var - r1) * (var - r2)
        lhs = lhs.expand()
        rhs = 0
    else:
        c1, c2, c3 = get_coefficients(3)
        lhs = c1 * var**2 + c2 * var + c3

    if rhs == None:
        c4, c5, c6 = get_coefficients(3, first_nonzero=False)
        rhs = c4 * var**2 + c5 * var + c6

    e = sympy.Eq(lhs, rhs)
    pvar = str(var)
    sols = ', '.join([pvar + " = " + sympy.latex(ex) for ex in sympy.solve(e, var)])
    sols = "$$" + sols + "$$"
    if len(sols) == 0:
        return make_quadratic_eq()
    return render(e), sols


def make_linear_eq(x="", rhs=None, var_coeffs=True):
    """
    Generates linear equation in one variable, and its solution.
    x : charector for the variable to be solved for. defaults to random selection
        from the global list `alpha`.
                            OR
        a list of possible charectors. A random selection will be made from them.
    rhs : value to set for the right-hand side. If not given, the 
          right-hand side will be a randomly generated linear expression
    var_coeffs : sets whether we want variables as coefficients in the problem.
                 defaults to True. Set to False if you want a problem with strictly
                 numerical coefficients.
    """
    if not x:
        x = random.choice(alpha)
    elif isinstance(x, list):
        x = random.choice(x)

    exclude = [x.upper(), x.lower()]
    x = sympy.Symbol(x)
    c1, c2, c3, c4 = get_coefficients(4, var_coeffs=var_coeffs, reduce=False,
                                      exclude=exclude)
    lhs = c1 * x + c2
    rhs = c3 * x + c4
    e = sympy.Eq(lhs, rhs)
    sols = [render(ex, x) for ex in sympy.solve(e, x)]
    return "Solve for $%s$ : %s" % (x, render(e)), sols


def make_rational_poly_simplify(var="x"):
    """
    Generates a rational expression of 4 polynomials, to be simplified.
    Example:
        ( (x**2 + 16*x + 60) / (x**2 - 36)) / 
        ( (x**2 - 2*x - 63) / (x**2 - 5*x - 36)
    x : charector for the variable to be solved for. defaults to random selection
        from the global list `alpha`.
                            OR
        a list of possible charectors. A random selection will be made from them.
    """
    if not var:
        var = random.choice(alpha)
    elif isinstance(var, list):
        var = random.choice(var)

    exclude = [var.upper(), var.lower()]
    x = sympy.Symbol(var)
    select = shuffle(range(-10, -1) + range(1, 10))[:6]
    e1 = sympy.prod([x - i for i in shuffle(select)[:2]]).expand()
    e2 = sympy.prod([x - i for i in shuffle(select)[:2]]).expand()
    e3 = sympy.prod([x - i for i in shuffle(select)[:2]]).expand()
    e4 = sympy.prod([x - i for i in shuffle(select)[:2]]).expand()
    L = len(set([e1, e2, e3, e4]))
    e = ((e1 / e2) / (e3 / e4))
    s1 = ''.join(["\\frac{", sympy.latex(e1), "}", "{", sympy.latex(e2), "}"])
    s2 = ''.join(["\\frac{", sympy.latex(e3), "}", "{", sympy.latex(e4), "}"])
    s3 = ''.join(["$$\\frac{", s1, "}", "{", s2, "}$$"])
    pieces = str(e.factor()).split("/")
    try:
        num, denom = [parse_expr(i).expand() for i in pieces]
    except:
        return make_rational_poly_simplify(var)
    if len(pieces) != 2 or L < 4 or degree(num) > 2 or degree(denom) > 2:
        return make_rational_poly_simplify(var)
    return s3, render(num / denom)

## Exponents Basics
### section 1
#### instruction : Simplify. Your answer should contain only positive exponents

-  easy
  - case 1
  - (int 2-5) ^ (int 2-5) $\cdot$ ( same int 2-5) ^ (int 2-5)
  - $ 4^2 \cdot 4^2 $
  - case 2
  - (int 2-5) (variable) ^ (int 2-5) $\cdot$ ( same variable) ^ (int 2-5)
  - $ 2x^5 \cdot x^2 $
  - case 3
  - (x^(int 2-5))^(int 2-5)
  - $ (x^2)^5 $
  - answer $ x^{10} $

-  medium
  - case 1
  - (int 2-5) ^ (int 2 to 5) $\cdot$ ( same int 2-5 ^ (int 2 to 5) $\cdot$ ( same int 2-5) ^ (int 5-10)
  - $ 4^2 \cdot 4^2 \cdot 4^5 $
  - answer $ 4^{9} $
  - case 2
  - (int 2-5) (variable) ^ (int 2 to 10) $\cdot$ (int 2-5)( same variable) ^ (int 2 to 10)
  - $ 2x^{5} \cdot 4x^{8} $
  - answer : $ 8x^{13} $
  - case 3
  - (int 2-5) (var1) ^ (int 2-5) $\cdot$ ( var1) ^ (int 2-5) $\cdot$ (int 2-5) (var2) ^ (int 2-5)
  - $ 2x^5 \cdot x^2 \cdot 4y^2$

-  hard
  - case 1
  - ((int 2-5) ^ (int 2 to 5) $\cdot$ ( same int 2-5 ^ (int 2 to 5) $\cdot$ ( same int 2-5) ^ (int 5-10)) ^(int 2)
  - $ (4^2 \cdot 4^2 \cdot 4^5)^2 $
  - answer $ 4^{18} $
  - case 2
  - (int 2-5) (variable) ^ (int 2 to 10) $\cdot$ ((int 2-5)( same variable) ^ (int 2 to 10))^2
  - $ 2x^{5} \cdot (4x^{8})^3 $
  - answer : $ 128x^{29} $
  - case 3
  - (int 2-5) (var1) ^ (int 2-5) (var2) ^ (int 2-5) $\cdot$ (int 2-5)( var1) ^ (int 2-5)(var2) ^ (int 2-5) $\cdot$ (int 2-5) (var2) ^ (int 2-5)
  - $ 2x^5y^2 \cdot 4x^2y^3 \cdot 4y^2$
  - answer $ 32x^7y^7$
  - case 4
  - ((int 2-5) (var1) ^ (int 2-5) (var2) ^ (int 2-5))^(int2-5) $\cdot$ (int 2-5)( var1) ^ (int 2-5)(var2) ^ (int 2-5) $\cdot$ (int 2-5) (var2) ^ (int 2-5)
  - $ (2x^2y^3 \cdot 2x^2y^2)^2 \cdot 3y^2$
  - answer $ 48x^8y^{12}$



In [ ]:
from sympy import symbols, sympify, latex
import random

def exponent_problem_generator(difficulty='easy', expr='latex' , option_1=2 , option_2=5 , option_3=2 , option_4=10):
  x,y,z,exp1,exp2 = symbols('x,y,z,exp1,exp2')
  varlist=['x','y','z']
  random.shuffle(varlist)
  var2 = varlist.pop()
  var1 = varlist.pop()
  intA = random.randint(option_1,option_2)
  intB = random.randint(option_1,option_2)
  intTEMP = random.randint(option_1,option_2)
  intTEMP2 = random.randint(option_1,option_2)

  intC = random.randint(option_3,option_4)
  intD = random.randint(option_3,option_4)

  if difficulty == 'easy':
    choices = random.randint(1,4)
    if choices == 1: # case1
      #expression = x**exp1 * x**exp2
      expression = fr'{intA}**{intB} * {intA}**{intTEMP}'
      expression_answer = fr'{intA}**{intB+intTEMP}'
      #answer = expression.subs([(x,intA) ,(exp1 , intB), (exp2 , intC)])
    elif choices == 2: # case2
      #expression = intC * x**exp1 * intD * x**exp2
      expression = fr'{intC}*{intA}**{intB} * {intD}*{intA}**{intTEMP}'
      expression_answer = fr'{intC*intD}*{intA}**{intB+intTEMP}'
      #answer = expression.subs([(x, intA) , (exp1 , intB) , (exp2 , intC)])
    elif choices == 3: # case3
      #expression = ( x**exp1 )**exp2
      expression = fr'({x}**{intA})**{intB}'
      expression_answer = fr'{x}**{intA*intB}'
      #answer = expression.subs([(x, intA) , (exp1 , intB) , (exp2 , intC)])
    elif choices == 4: # case3
      #expression = ( x**exp1 )**exp2
      expression = fr'({intTEMP}**{intA})**{intB}'
      expression_answer = fr'{intTEMP}**{intA*intB}'
      #answer = expression.subs([(x, intA) , (exp1 , intB) , (exp2 , intC)])
  elif difficulty == 'medium':
    choices = random.randint(1,3)
    if choices == 1: # case1
      expression = fr'{intA}**{intTEMP2} * {intA}**{intB} * {intA}**{intTEMP}'
      expression_answer = fr'{intA}**{intTEMP2+intTEMP+intB}'
    elif choices == 2: # case2
      expression = fr'{intC}*{intA}**{intTEMP2} * {intD}*{intA}**{intTEMP}'
      expression_answer = fr'{intC*intD}*{intA}**{intTEMP2+intTEMP}'
    elif choices == 3: # case3
      expression = fr'{intC}*{var1}**{intB} * {intD}*{var1}**{intTEMP} * {intA} *{var2}**{intB}'
      expression_answer = fr'{intC*intD*intA}*{var2}**{intTEMP}*{var1}**{intB+intTEMP}'
  elif difficulty == 'hard':
    choices = random.randint(1,3)
    if choices == 1: # case1
      expression = fr'({intA}**{intTEMP2} * {intA}**{intB} * {intA}**{intTEMP})**{intC}'
      expression_answer = fr'{intA}**{(intTEMP2+intTEMP+intB)*intC}'
    elif choices == 2: # case2
      expression = fr'{intC}*{intA}**{intTEMP2} * ({intD}*{intA}**{intTEMP})**{intB}'
      expression_answer = fr'{intC* intD**intB}*{intA}**{intTEMP2+intTEMP*intB}'
    elif choices == 3: # case3   
      expression = fr'({intC}*{var1}**{intB} * {intA}*{var1}**{intTEMP})**{intTEMP2} * {intA} *{var2}**{intB}'
      expression_answer = fr'{intC**intTEMP2 * intA**intTEMP2 *intA}*{var2}**{intTEMP}*{var1}**{intB*intTEMP2+intTEMP*intTEMP2}'
  if expr =='latex':
    problem = '$'+latex(sympify(expression, evaluate = False))+'$'
    answer = '$'+latex(sympify(expression_answer, evaluate = False))+'$'
  else:
    problem = sympify(expression, evaluate = False)
    answer = sympify(expression_answer, evaluate = False)
  return problem , answer




for i in range(10):
  a,b = exponent_problem_generator('easy')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(10):
  a,b = exponent_problem_generator('medium')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(10):
  a,b = exponent_problem_generator('hard')
  print( 'problem',a , 'answer',b)
  print('\\newline ')


problem $\left(5^{5}\right)^{3}$ answer $5^{15}$
\newline 
problem $\left(x^{3}\right)^{5}$ answer $x^{15}$
\newline 
problem $\left(x^{2}\right)^{4}$ answer $x^{8}$
\newline 
problem $\left(x^{2}\right)^{2}$ answer $x^{4}$
\newline 
problem $\left(4^{4}\right)^{4}$ answer $4^{16}$
\newline 
problem $9 \cdot 5^{2} \cdot 7 \cdot 5^{4}$ answer $63 \cdot 5^{6}$
\newline 
problem $2 \cdot 3^{5} \cdot 9 \cdot 3^{4}$ answer $18 \cdot 3^{9}$
\newline 
problem $\left(x^{5}\right)^{4}$ answer $x^{20}$
\newline 
problem $4^{4} \cdot 4^{5}$ answer $4^{9}$
\newline 
problem $2^{2} \cdot 2^{4}$ answer $2^{6}$
\newline 
problem $2 \cdot 5^{5} \cdot 10 \cdot 5^{2}$ answer $20 \cdot 5^{7}$
\newline 
problem $5^{2} \cdot 5^{3} \cdot 5^{5}$ answer $5^{10}$
\newline 
problem $6 \cdot 2^{2} \cdot 5 \cdot 2^{5}$ answer $30 \cdot 2^{7}$
\newline 
problem $10 \cdot 5^{3} \cdot 9 \cdot 5^{3}$ answer $90 \cdot 5^{6}$
\newline 
problem $3^{3} \cdot 3^{3} \cdot 3^{5}$ answer $3^{11}$
\newline 
problem $3^{3} \cd

## Exponents Division
### section 1
#### instruction : Simplify. Your answer should contain only positive exponents

-  easy
  - case 1
  - var1^(int2-5) / var1^(int2-5)
  - problem : $ \frac{x^2}{x^5} $
  - answer: $ \frac{1}{x^3} $
  - case 2
  - (int2-5)var1^(int2-5) / (int2-10)var1^(int2-5)
  - problem : $ \frac{2x^2}{10x^5} $
  - answer: $ \frac{1}{5x^3} $

-  medium
  - case 1
  - (int2-10) var1^(int5-10) / (int2-10)var1^(int5-10)
  - problem: $ \frac{5x^8}{8x^6} $
  - answer: $ \frac{5x^2}{8} $
  - case 2
  - (int2-5)var1^(int2-5)var2^(int2-5) / (int2-10)var1^(int2-5)var2^(int2-5)
  - problem :$ \frac{2x^2y^5}{10x^5y^2} $
  - answer: $ \frac{y^3}{5x^3} $

-  hard
  - case 1
  - (int5-20)var1^(int2-10)var2^(int2-10) / (int5-30)var1^(int2-10)var2^(int2-10)
  - problem $ \frac{18x^8y^10}{10x^5y^2} $
  - answer: $ \frac{9x^3y^3}{5} $
  - case 2
  - (int5-20)var1^(int2-10)var2^(int2-10)var3^(int2-10) / (int5-30)var1^(int2-10)var2^(int2-10)var3^(int2-10)
  - problem : $ \frac{2x^2y^5z^8}{10x^5y^2z^{10}} $
  - answer: $ \frac{y^3}{5x^3z^2} $

In [ ]:
from sympy import symbols
import random

def Exponent_Division(difficulty='easy', expr='latex' , option_1=2 , option_2=10 , option_3=2 , option_4=20):
  x,y,z,a,b,c = symbols('x,y,z,a,b,c')
  varlist=['x','y','z','a','b','c']
  random.shuffle(varlist)
  var1 = varlist.pop()
  var2 = varlist.pop()
  var3 = varlist.pop()
  intA = random.randint(option_1,option_2)
  intB = random.randint(option_1,option_2)
  intC = random.randint(option_1,option_2)
  intD = random.randint(option_1,option_2)

  intE = random.randint(option_3,option_4)
  intF = random.randint(option_3,option_4)
  intG = random.randint(option_3,option_4)
  intH = random.randint(option_3,option_4)
  intI = random.randint(option_3,option_4)
  intJ = random.randint(option_3,option_4)


  if difficulty == 'easy':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'{var1}**{intB} / {var1}**{intD}'
    elif choices == 2: # case2
      expression = fr'({intE}*{var2}**{intB}) / ({intF}*{var2}**{intD})'
  elif difficulty == 'medium':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intG}*{var1}**{intB} )/ ({intH}*{var1}**{intD})'
    elif choices == 2: # case2
      expression = fr'({intE}*{var2}**{intB}) / ({intF}*{var2}**{intD})'
  elif difficulty == 'hard':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intG}*{var1}**{intB}*{var2}**{intC} )/ ({intH}*{var1}**{intD}*{var2}**{intA})'
    elif choices == 2: # case2
      expression = fr'({intE}*{var2}**{intB}*{var1}**{intG}) / ({intF}*{var2}**{intD}*{var1}**{intH})'
    elif choices == 3: # case3
      expression = fr'({intE}*{var2}**{intB}*{var1}**{intG}*{var3}**{intI}) / ({intF}*{var2}**{intD}*{var1}**{intH}*{var3}**{intJ})'


  if expr =='latex':
    problem = '$'+latex(sympify(expression, evaluate = False))+'$'
    answer = '$'+latex(sympify(expression))+'$'
  else:
    problem = sympify(expression, evaluate = False)
    answer = sympify(expression)
  return problem , answer
'''
for i in range(10):
  a,b = Exponent_Division('easy')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(10):
  a,b = Exponent_Division('medium')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(10):
  a,b = Exponent_Division('hard')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
'''

"\nfor i in range(10):\n  a,b = Exponent_Division('easy')\n  print( 'problem',a , 'answer',b)\n  print('\\newline ')\nfor i in range(10):\n  a,b = Exponent_Division('medium')\n  print( 'problem',a , 'answer',b)\n  print('\\newline ')\nfor i in range(10):\n  a,b = Exponent_Division('hard')\n  print( 'problem',a , 'answer',b)\n  print('\\newline ')\n"

## Powers of Products
### section 1
#### instruction : Simplify. Your answer should contain only positive exponents

-  easy
  - case 1
  - (int(2-10) (var1)^(int2-10))^(int2-10)
  - problem : $ (2 x^{7})^7 $
  - answer: $ 2^{7}x^{49} $
  - case 2
  - (int(2-10)^(int2-5) (var1)^(int2-10))^(int2-10)
  - problem : $ (2^3 x^{7})^5 $
  - answer: $ 2^{15}x^{35} $

-  medium
  - case 1
  - (int(2-10) (var1)^(int2-10) (var2)^(int2-10))^(int2-10)
  - problem : $ (2 x^{7}y^{3})^7 $
  - answer: $ 2^{7}x^{49}y^{21} $
  - case 2
  - (int(2-10)^(int2-5) (var1)^(int2-10)(var2)^(int2-10))^(int2-10)
  - problem : $ (2^{3} x^{7}y^{2})^5 $
  - answer: $ 2^{15}x^{35}y^{10}$


-  hard
  - case 1
  - (int(2-10) (var1)^(int2-10) (var2)^(int2-10) int(2-10) (var1)^(int2-10) (var2)^(int2-10))^(int2-10)
  - problem : $ (2 x^{3}y^{4}\cdot 7 x^{2}y^{5})^7 $
  - answer: $ 2^{7}7^{7}x^{35}y^{63} $


In [ ]:
#fix number bases gathering problems
from sympy import symbols,latex,sympify,powsimp
import random
import re

def isprime(n):
    return re.compile(r'^1?$|^(11+)\1+$').match('1' * n) is None

def Powers_Of_Products(difficulty='easy', expr='latex' , option_1=2 , option_2=10 , option_3=2 , option_4=20):
  '''
  option_1 = int min
  option_2 = int max
  option_3 = int min
  option_4 = int max
  '''
  x,y,z,a,b,c = symbols('x,y,z,a,b,c',commutative=False)
  varlist=['x','y','z','a','b','c']
  random.shuffle(varlist)
  var1 = varlist.pop()
  var2 = varlist.pop()
  var3 = varlist.pop()

  primeNumbers = [x for x in range(100) if isprime(x)]

  intA = random.randint(option_1,option_2)
  intB = random.randint(option_1,option_2)
  intC = random.randint(option_1,option_2)
  intD = random.randint(option_1,option_2)

  intE = random.randint(option_3,option_4)
  intF = random.randint(option_3,option_4)
  intG = random.randint(option_3,option_4)
  intH = random.randint(option_3,option_4)
  intI = random.randint(option_3,option_4)
  intJ = random.randint(option_3,option_4)


  if difficulty == 'easy':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intA} * {var1}**{intB} )**{intD}'
      expression_answer = fr'{intA}**{intD} * {var1}**({intB*intD}) '
    elif choices == 2: # case2
      expression = fr'({intA}**{intB} * {var1}**{intC} )**{intD}'
      expression_answer = fr'{intA}**({intB*intD}) * {var1}**({intC*intD})'

  elif difficulty == 'medium':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intA} * {var1}**{intB} *{var2}**{intC})**{intD}'
      expression_answer = fr'{intA}**{intD} * {var1}**{intB*intD}*{var2}**{intC*intD} '
    elif choices == 2: # case2
      expression = fr'({intA}**{intB} * {var1}**{intC} *{var2}**{intD})**{intE}'
      expression_answer = fr'{intA}**{intB+intE} * {var1}**{intC*intE} * {var2}**{intE*intD}'

  elif difficulty == 'hard':
    expression = fr'({intA}**{intB} * {var1}**{intC} *{var2}**{intD}* {intA}**{intE} * {var2}**{intF} *{var1}**{intG})**{intI}'
    expression_answer = fr'{intA}**{intB*intI+intE*intI} * {var1}**{intC*intI + intG*intI} *{var2}**{intD*intI+intF*intI}'


  if expr =='latex':
    problem = '$'+latex(sympify(expression, evaluate = False))+'$'
    answer = '$'+latex(sympify(expression_answer, evaluate = False))+'$'
  else:
    problem = sympify(expression, evaluate = False)
    answer = sympify(expression_answer, evaluate = False)
  return problem , answer

for i in range(5):
  a,b = Powers_Of_Products('easy')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(5):
  a,b = Powers_Of_Products('medium')
  print( 'problem',a , 'answer',b)
  print('\\newline ')
for i in range(10):
  a,b = Powers_Of_Products('hard')
  print( 'problem',a , 'answer',b)
  print('\\newline ')



problem $\left(2 c^{7}\right)^{4}$ answer $2^{4} c^{28}$
\newline 
problem $\left(7 b^{3}\right)^{4}$ answer $7^{4} b^{12}$
\newline 
problem $\left(7 y^{7}\right)^{4}$ answer $7^{4} y^{28}$
\newline 
problem $\left(3 a^{10}\right)^{5}$ answer $3^{5} a^{50}$
\newline 
problem $\left(4 y^{4}\right)^{6}$ answer $4^{6} y^{24}$
\newline 
problem $\left(6 a^{6} x^{7}\right)^{9}$ answer $6^{9} a^{54} x^{63}$
\newline 
problem $\left(2^{7} x^{7} y^{7}\right)^{4}$ answer $2^{11} x^{28} y^{28}$
\newline 
problem $\left(5 a^{9} y^{2}\right)^{5}$ answer $5^{5} a^{45} y^{10}$
\newline 
problem $\left(10 a^{4} b^{9}\right)^{3}$ answer $10^{3} a^{12} b^{27}$
\newline 
problem $\left(2^{9} a^{3} x^{8}\right)^{18}$ answer $2^{27} a^{54} x^{144}$
\newline 
problem $\left(2^{10} \cdot 2^{15} b^{3} b^{8} x^{3} x^{5}\right)^{18}$ answer $2^{450} b^{198} x^{144}$
\newline 
problem $\left(2^{4} \cdot 2^{18} c^{3} c^{20} x^{4} x^{10}\right)^{5}$ answer $2^{110} c^{115} x^{70}$
\newline 
problem $\left(6^{8} 

## Scientific Notation 
[link text](https://python-reference.readthedocs.io/en/latest/docs/float/scientific.html)

### use numpy [link text](https://www.delftstack.com/howto/python/scientific-notation-python/)

### section 1
#### instruction : Write each number in scientific notation.

-  easy (Whole only)
  - case 1
  - 100~10000
  - problem : $ 8888 $
  - answer: $ 8.888 \times 10^{3} $

-  medium (decimal only)
  - case 1
  - 0.0001 - 0.9999
  - problem : $ 0.1234 $
  - answer: $ 1.234 \times 10^{-1} $

-  hard (Whole and decimal)
  - case 1
  - 0.00000001 - 999999.999999
  - problem : $ 0.00001490 $
  - answer: $ 1.49 \times 10^{-5} $

### section 2
#### instruction : Write each number in standard notation.

-  easy (Whole only)
  - case 1
  - 100~10000
  - problem : $ 8.888 \times 10^{3} $
  - answer: $ 8888 $

-  medium (decimal only)
  - case 1
  - 0.0001 - 0.9999
  - problem : $ 1.234 \times 10^{-1} $
  - answer: $ 0.1234 $

-  hard (Whole and decimal)
  - case 1
  - 0.00000001 - 999999.999999
  - problem : $ 1.49 \times 10^{-5} $
  - answer: $ 0.00001490 $

### section 3
#### instruction : Write each number in scientific notation.

-  easy (Whole only)
  - case 1
  - (randfloat5 - 100) | power is $\pm$ random(2,5)
  - problem : $ 12 \times 10^3 $
  - answer: $ 1.2 \times 10^2 $

-  medium (decimal only)
  - case 1
  - (randfloat.1 - .9999) | power is $\pm$ random(2,5)
  - problem : $ 0.99\times 10^{4} $
  - answer: $ 0.9999\times 10^{2} $

-  hard (Whole and decimal)
  - case 1
  - (randfloat.00001 - 9999.99999) | power is $\pm$ random(2,10)
  - problem : $ 9999.99012\times 10^{4} $
  - answer: $ 9.999999012\times 10^{8} $

### section 4
#### instruction : Simplify. Write each answer in scientific notation.

-  easy
  - case 1
  - (randfloat1 - 100) $\times 10^{int2-5}$ (randfloat1 - 100) $\times 10^{int2-5}$ 
  - problem : $ (0.12 \times 10^3) (0.12 \times 10^3)$
  - answer: $ 1.44 \times 10^4 $

-  medium
  - case 1
  - (randfloat.1 - .999) $\times 10^{int2-10}$ (randfloat.1 - .999) $\times 10^{int2-10}$ 
  - problem : $ (0.112 \times 10^3) (0.972 \times 10^8)$
  - answer: $ 1.08864 \times 10^9 $

-  hard
  - case 1
  - (randfloat.001 - 99.99) $\times 10^{int2-10}$ (randfloat.001 - 99.99) $\times 10^{int2-10}$ 
  - problem : $ (12.11 \times 10^2) (72.72 \times 10^3)$
  - answer: $ 8.806392 \times 10^7 $

In [ ]:
import random

def insert_comma(num):
    num = int(num)
    return (f"{num:,}")

def insert_comma2(num):
    num = float(num)
    return (f"{num:,}")

def truncate(number):
    number = float(number)
    return((number * 100) / 100)

def decimal_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]

    return(problem)

def decimal_only_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]
    elif float(problem) == 0.0:
        decimal_only_generator(option_difficulty)
    return(problem)

def Scientific_Notation_Section_1(option_difficulty = 'easy', expr= 'latex'):
    one = ""
    answer = ""
    squared = ""
    if option_difficulty == "easy":
        one = random.randint(100,10000)

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        squared = answer[answer.find("+"):]
        squared = int(squared)
        answer = answer[:answer.find("+") + 1]
        answer = answer.replace("E+", r" \times 10^{"+ str(squared) +"}")

        return(insert_comma(one), answer)

    elif option_difficulty == "medium":
        squared = ""
        one = float(decimal_generator(option_difficulty))

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))


        squared = answer[answer.find("-"):]
        squared = int(str(squared))
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared)+"}")

        return(one, answer)
    
    elif option_difficulty == "hard":
        number = ""
        decimal = ""
        number = random.randint(0, 999999)
        decimal = float(decimal_only_generator(option_difficulty))
        one = float(number + decimal)

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared = answer[answer.find("+"):]
            squared = int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared)+"}")
        else:
            squared = answer[answer.find("-"):]
            squared = int(str(squared))
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared)+"}")

        return(insert_comma2(one), answer)

problem, answer = Scientific_Notation_Section_1("easy", "latex")
print("easy")
print(problem, answer)
problem, answer = Scientific_Notation_Section_1("medium", "latex")
print("medium")
print(problem, answer)
problem, answer = Scientific_Notation_Section_1("hard", "latex")
print("hard")
print(problem, answer)


easy
9,460 9.46 \times 10^{3}
medium
0.478 4.78 \times 10^{-1}
hard
781,635.08763636 7.8163508764 \times 10^{5}


In [ ]:
import random

def insert_comma(num):
    num = int(num)
    return (f"{num:,}")

def insert_comma2(num):
    num = float(num)
    return (f"{num:,}")

def truncate(number):
    number = float(number)
    return((number * 100) / 100)

def decimal_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.5f}".format(decimal)
        else:
            problem = "{:.6f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]

    return(problem)

def decimal_only_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]
    elif float(problem) == 0.0:
        decimal_only_generator(option_difficulty)
    return(problem)

def Scientific_Notation_Section_2(option_difficulty = 'easy', expr= 'latex'):
    one = ""
    answer = ""
    squared = ""
    if option_difficulty == "easy":
        one = random.randint(100,10000)

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))
        
        squared = answer[answer.find("+"):]
        squared = int(squared)
        answer = answer[:answer.find("+") + 1]
        answer = answer.replace("E+", r" \times 10^{"+ str(squared)+"}")

        return(answer, insert_comma(one))

    elif option_difficulty == "medium":
        squared = ""
        one = float(decimal_generator(option_difficulty))

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        squared = answer[answer.find("-"):]
        squared = int(str(squared))
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared)+"}")

        return(answer, one)
    
    elif option_difficulty == "hard":
        number = ""
        decimal = ""
        number = random.randint(0, 999999)
        decimal = float(decimal_only_generator(option_difficulty))
        one = float(number + decimal)

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared = answer[answer.find("+"):]
            squared = int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared)+"}")
        else:
            squared = answer[answer.find("-"):]
            squared = int(str(squared))
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared)+"}")

        return(answer, insert_comma2(one))

problem, answer = Scientific_Notation_Section_2("easy", "latex")
print("easy")
print(problem, answer)
problem, answer = Scientific_Notation_Section_2("medium", "latex")
print("medium")
print(problem, answer)
problem, answer = Scientific_Notation_Section_2("hard", "latex")
print("hard")
print(problem, answer)


easy
9.654 \times 10^{3} 9,654
medium
8.36 \times 10^{-1} 0.836
hard
8.1948918021 \times 10^{5} 819,489.1802052


In [ ]:
import random

def insert_comma(num):
    num = int(num)
    return (f"{num:,}")

def insert_comma2(num):
    num = float(num)
    return (f"{num:,}")

def truncate(number):
    number = float(number)
    return((number * 100) / 100)

def decimal_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.6f}".format(decimal)
        else:
            problem = "{:.5f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]

    return(problem)

def decimal_only_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]
    elif float(problem) == 0.0:
        decimal_only_generator(option_difficulty)
    return(problem)

def Scientific_Notation_Section_3(option_difficulty = 'easy', expr= 'latex'):
    one = ""
    problem = ""
    answer = ""
    squared = ""
    squared_2 = ""
    negative = ["-", ""]

    if option_difficulty == "easy": 
        one = random.randint(5, 100)
        squared = str(random.choice(negative)) + str(random.randint(2,5))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared_2 = answer[answer.find("+"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
        else:
            squared_2 = answer[answer.find("-"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)

    elif option_difficulty == "medium":
        one = float(decimal_generator(option_difficulty))
        squared = str(random.choice(negative)) + str(random.randint(2,5))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}') 
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        squared_2 = answer[answer.find("-"):]
        squared_2 = int(squared_2) + int(squared)
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)
    
    elif option_difficulty == "hard": 
        number = ""
        decimal = ""
        number = random.randint(0, 9999)
        decimal = float(decimal_only_generator("medium"))
        squared = str(random.choice(negative)) + str(random.randint(2,10))
        one = float(float(number) + float(decimal))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared_2 = answer[answer.find("+"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
        else:
            squared_2 = answer[answer.find("-"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)

problem, answer = Scientific_Notation_Section_3("easy", "latex")
print("easy")
print(problem, answer)

problem, answer = Scientific_Notation_Section_3("medium", "latex")
print("medium")
print(problem, answer)

problem, answer = Scientific_Notation_Section_3("hard", "latex")
print("hard")
print(problem, answer)


easy
5 \times 10^{-3} 5.0 \times 10^{-3}
medium
0.9127 \times 10^{3} 9.127 \times 10^{2}
hard
1933.824 \times 10^{3} 1.933824 \times 10^{6}


In [ ]:
import random

def insert_comma(num):
    num = int(num)
    return (f"{num:,}")

def insert_comma2(num):
    num = float(num)
    return (f"{num:,}")

def truncate(number):
    number = float(number)
    return((number * 100) / 100)

def decimal_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.6f}".format(decimal)
        else:
            problem = "{:.5f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]

    return(problem)

def decimal_only_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]
    elif float(problem) == 0.0:
        decimal_only_generator(option_difficulty)
    return(problem)

def Scientific_Notation_Section_3(option_difficulty = 'easy', expr= 'latex'):
    one = ""
    problem = ""
    answer = ""
    squared = ""
    squared_2 = ""
    negative = ["-", ""]

    if option_difficulty == "easy": 
        one = random.randint(5, 100)
        squared = str(random.choice(negative)) + str(random.randint(2,5))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared_2 = answer[answer.find("+"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
        else:
            squared_2 = answer[answer.find("-"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)

    elif option_difficulty == "medium":
        one = float(decimal_generator(option_difficulty))
        squared = str(random.choice(negative)) + str(random.randint(2,5))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}') 
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        squared_2 = answer[answer.find("-"):]
        squared_2 = int(squared_2) + int(squared)
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)
    
    elif option_difficulty == "hard": 
        number = ""
        decimal = ""
        number = random.randint(0, 9999)
        decimal = float(decimal_only_generator("medium"))
        squared = str(random.choice(negative)) + str(random.randint(2,10))
        one = float(float(number) + float(decimal))

        problem = str(one) + r" \times 10^{"+ str(squared)+"}"

        answer = str(f'{one: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        if answer.find("+") != -1:
            squared_2 = answer[answer.find("+"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("+") + 1]
            answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
        else:
            squared_2 = answer[answer.find("-"):]
            squared_2 = int(squared_2) + int(squared)
            answer = answer[:answer.find("-") + 1]
            answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")

        return(problem, answer)

problem, answer = Scientific_Notation_Section_3("easy", "latex")
print("easy")
print(problem, answer)

problem, answer = Scientific_Notation_Section_3("medium", "latex")
print("medium")
print(problem, answer)

problem, answer = Scientific_Notation_Section_3("hard", "latex")
print("hard")
print(problem, answer)


easy
69 \times 10^{3} 6.9 \times 10^{4}
medium
0.111 \times 10^{2} 1.11 \times 10^{1}
hard
9318.071 \times 10^{-8} 9.318071 \times 10^{-5}


In [ ]:
import random

def insert_comma(num):
    num = int(num)
    return (f"{num:,}")

def insert_comma2(num):
    num = float(num)
    return (f"{num:,}")

def truncate(number):
    number = float(number)
    return((number * 100) / 100)

def decimal_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.6f}".format(decimal)
        else:
            problem = "{:.5f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]

    return(problem)

def decimal_only_generator(option_difficulty):
    problem = ""
    integer_only = 0
    decimal = 0.0
    decimal_2 = 0.0
    if option_difficulty == "easy": # get rid of 74.90 <--- problem!!! (EDGE 0)
        integer_only = random.randint(1, 9)
        decimal = random.random()
        if integer_only % 2 == 0: 
            problem = "{:.1f}".format(decimal)
        else:
            problem = "{:.2f}".format(decimal)

    elif option_difficulty == "medium":
        integer_only = random.randint(1, 99)
        decimal = random.random()
        decimal_2 = "{:.2f}".format(decimal)
        if integer_only % 2 == 0:
            if (float(decimal_2) * 100) % 2 == 0:
                problem  = "{:.1f}".format(float(decimal))
            elif int(str(decimal_2 * 100)[:1]) % 3 == 0:
                problem  = "{:.4f}".format(float(decimal))
            else:
                problem = "{:.2f}".format(float(decimal))
        else: #add one more (ten thousandth place) if the front is divisible by 2
            problem = "{:.3f}".format(float(decimal))

    elif option_difficulty == "hard":
        integer_only = random.randint(10, 100)
        decimal = random.random()
        if integer_only % 2 == 0:
            problem = "{:.8f}".format(decimal)
        else:
            problem = "{:.7f}".format(decimal)
            #will have two cutt off points
    
    if str(problem)[-2] == "." and str(problem)[-1] == "0":
        problem = problem[:-2]
    elif float(problem) == 0.0:
        decimal_only_generator(option_difficulty)
    return(problem)

def sci_formatter(answer, squared):
    squared_2 = ""
    if answer.find("+") != -1:
        squared_2 = answer[answer.find("+"):]
        squared_2 = int(squared_2) + int(squared)
        answer = answer[:answer.find("+") + 1]
        answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
    else:
        squared_2 = answer[answer.find("-"):]
        squared_2 = int(squared_2) + int(squared)
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")
    return(answer, float(answer[:answer.find("t") - 2]), answer[answer.find("{") +1:-1])


def sciformatter(answer, squared):
    squared_2 = ""
    if answer.find("+") != -1:
        squared_2 = int(squared)
        answer = answer[:answer.find("+") + 1]
        answer = answer.replace("E+", r" \times 10^{"+ str(squared_2)+"}")
    else:
        squared_2 = int(squared)
        answer = answer[:answer.find("-") + 1]
        answer = answer.replace("E-", r" \times 10^{"+ str(squared_2)+"}")
    return(answer, float(answer[:answer.find("t") - 2]))



def Scientific_Notation_Section_4(option_difficulty = 'easy', expr= 'latex'):
    one = ""
    two = ""
    problem = ""
    answer = ""
    answer_1 = ""
    answer_2 = ""
    squared = ""
    squared_two = ""
    squared_2 = ""
    negative = ["-", ""]
    filler = ""

    if option_difficulty == "easy": # (7 × 10^2)(9 × 10^6) solve then scientific
        first_para = ""
        second_para = ""
        one = random.randint(5, 100)
        two = random.randint(5, 100)
        squared = str(random.choice(negative)) + str(random.randint(2,5))
        squared_two = str(random.choice(negative)) + str(random.randint(2,5))

        answer_1 = str(f'{one: .10E}')
        answer_1 = str(round(truncate(float(answer_1[:answer_1.find("E")])), 9)) + str(answer_1[answer_1.find("E"):])

        answer_2 = str(f'{two: .10E}')
        answer_2 = str(round(truncate(float(answer_2[:answer_2.find("E")])), 9)) + str(answer_2[answer_2.find("E"):])

        first_para, one, squared = sci_formatter(answer_1, squared)
        second_para, two, squared_two = sci_formatter(answer_2, squared_two)

        problem = fr'({first_para})({second_para})'

        answer = (one) * (two)

        answer = str(f'{answer: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        answer = sciformatter(answer, int(squared) + int(squared_two) + 1)[0]
        return(problem, answer)


    elif option_difficulty == "medium":
        first_para = ""
        second_para = ""
        one = float(decimal_generator(option_difficulty))
        two = float(decimal_generator(option_difficulty))
        squared = str(random.choice(negative)) + str(random.randint(2,5))
        squared_two = str(random.choice(negative)) + str(random.randint(2,5))

        answer_1 = str(f'{one: .10E}')
        answer_1 = str(round(truncate(float(answer_1[:answer_1.find("E")])), 9)) + str(answer_1[answer_1.find("E"):])

        answer_2 = str(f'{two: .10E}')
        answer_2 = str(round(truncate(float(answer_2[:answer_2.find("E")])), 9)) + str(answer_2[answer_2.find("E"):])

        first_para, one, squared = sci_formatter(answer_1, squared)
        second_para, two, squared_two = sci_formatter(answer_2, squared_two)

        problem = fr'({first_para})({second_para})'

        answer = (one) * (two)

        answer = str(f'{answer: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        answer = sciformatter(answer, int(squared) + int(squared_two) + 1)[0]
        return(problem, answer)

    elif option_difficulty == "hard":
        first_para = ""
        second_para = ""

        one = float(str(random.randint(5, 99)) + str(decimal_generator("medium")))
        two = float(str(random.randint(5, 99)) + str(decimal_generator("medium")))

        squared = str(random.choice(negative)) + str(random.randint(2,10))
        squared_two = str(random.choice(negative)) + str(random.randint(2,10))

        answer_1 = str(f'{one: .10E}')
        answer_1 = str(round(truncate(float(answer_1[:answer_1.find("E")])), 9)) + str(answer_1[answer_1.find("E"):])

        answer_2 = str(f'{two: .10E}')
        answer_2 = str(round(truncate(float(answer_2[:answer_2.find("E")])), 9)) + str(answer_2[answer_2.find("E"):])

        first_para, one, squared = sci_formatter(answer_1, squared)
        second_para, two, squared_two = sci_formatter(answer_2, squared_two)

        problem = fr'({first_para})({second_para})'

        answer = (one) * (two)

        answer = str(f'{answer: .10E}')
        answer = str(str(round(float(truncate(answer[:answer.find("E")])), 9)) + str(answer[answer.find("E"):]))

        answer = sciformatter(answer, int(squared) + int(squared_two) + 1)[0]
        return(problem, answer)

problem, answer = Scientific_Notation_Section_4("easy", "latex")
print("easy")
print(problem, answer)

problem, answer = Scientific_Notation_Section_4("medium", "latex")
print("medium")
print(problem, answer)

problem, answer = Scientific_Notation_Section_4("hard", "latex")
print("hard")
print(problem, answer)


easy
(5.0 \times 10^{4})(7.1 \times 10^{-1}) 3.55 \times 10^{4}
medium
(9.08 \times 10^{4})(4.0 \times 10^{-4}) 3.632 \times 10^{1}
hard
(2.40045 \times 10^{-3})(2.50504 \times 10^{10}) 6.013223268 \times 10^{8}


In [ ]:
import random
import numpy as np

def precision_round(number, digits=3):
    power = "{:e}".format(number).split('e')[1]
    return round(number, -(int(power) - digits))

digitEasy = random.randint(1,2)
digitMedium = random.randint(3,5)
digitHard = random.randint(6,8)

range_min=random.randint(0,9)
range_max=random.randint(100,999)

floatA = round(random.uniform(range_min,range_max), digitHard)
floatB = round(random.uniform(range_min,range_max), digitHard)


print(floatA, floatB)
print(format(floatA,'.1E'))

print(precision_round(1.234567))

np.format_float_scientific(np.float32(np.pi))
s = np.float32(1.23e24)
no1 = 1234567.1234567
print(np.format_float_scientific(s, unique=False, precision=15))
print(np.format_float_scientific(s, exp_digits=4))
print(np.format_float_scientific(no1, precision = 1, exp_digits=3))
print(np.format_float_scientific(no1, precision = 2, exp_digits=3))

fraction = 112345123123123.12345678
print(f'{fraction:e}')
decimal_places = int(f'{fraction:e}'.split('e')[-1])
print(decimal_places)

293.35981 408.532429
2.9E+02
1.235
1.230000071797338e+24
1.23e+0024
1.2e+006
1.23e+006
1.123451e+14
14


In [ ]:
#fix number bases gathering problems
from sympy import symbols,latex,sympify,powsimp
import random
import re

def isprime(n):
    return re.compile(r'^1?$|^(11+)\1+$').match('1' * n) is None

def Powers_Of_Products(difficulty='easy', expr='latex' , option_1=2 , option_2=10 , option_3=2 , option_4=20):
  '''
  option_1 = int min
  option_2 = int max
  option_3 = int min
  option_4 = int max
  '''
  x,y,z,a,b,c = symbols('x,y,z,a,b,c',positive = True,real=True)
  varlist=['x','y','z','a','b','c']
  random.shuffle(varlist)
  var1 = varlist.pop()
  var2 = varlist.pop()
  var3 = varlist.pop()

  primeNumbers = [x for x in range(100) if isprime(x)]

  intA = random.randint(option_1,option_2)
  intB = random.randint(option_1,option_2)
  intC = random.randint(option_1,option_2)
  intD = random.randint(option_1,option_2)

  intE = random.randint(option_3,option_4)
  intF = random.randint(option_3,option_4)
  intG = random.randint(option_3,option_4)
  intH = random.randint(option_3,option_4)
  intI = random.randint(option_3,option_4)
  intJ = random.randint(option_3,option_4)
  if difficulty == 'easy':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intA} * {var1}**{intB} )**{intD}'
      expression_answer = fr'{intA}**{intD} * {var1}**({intB*intD}) '
    elif choices == 2: # case2
      expression = fr'({intA}**{intB} * {var1}**{intC} )**{intD}'
      expression_answer = fr'{intA}**({intB*intD}) * {var1}**({intC*intD})'

  elif difficulty == 'medium':
    choices = random.randint(1,2)
    if choices == 1: # case1
      expression = fr'({intA} * {var1}**{intB} *{var2}**{intC})**{intD}'
      expression_answer = fr'{intA}**{intD} * {var1}**{intB*intD}*{var2}**{intC*intD} '
    elif choices == 2: # case2
      expression = fr'({intA}**{intB} * {var1}**{intC} *{var2}**{intD})**{intE}'
      expression_answer = fr'{intA}**{intB+intE} * {var1}**{intC*intE} * {var2}**{intE*intD}'

  elif difficulty == 'hard':
    expression = fr'({intA}**{intB} * {var1}**{intC} *{var2}**{intD}* {intA}**{intE} * {var2}**{intF} *{var1}**{intG})**{intI}'
    expression_answer = fr'{intA}**{intB*intI+intE*intI} * {var1}**{intC*intI + intG*intI} *{var2}**{intD*intI+intF*intI}'


  if expr =='latex':
    problem = '$'+latex(sympify(expression, evaluate = False))+'$'
    answer = '$'+latex(sympify(expression_answer, evaluate = False))+'$'
  else:
    problem = sympify(expression, evaluate = False)
    answer = sympify(expression_answer, evaluate = False)
  return problem , answer

for i in range(5):
  a,b = Powers_Of_Products('easy')
  print( 'problem',a , 'answer',b)
  print('\\newline ')

problem $\left(3^{9} c^{8}\right)^{6}$ answer $3^{54} c^{48}$
\newline 
problem $\left(7 b^{7}\right)^{4}$ answer $7^{4} b^{28}$
\newline 
problem $\left(7^{4} x^{4}\right)^{2}$ answer $7^{8} x^{8}$
\newline 
problem $\left(8 y^{4}\right)^{10}$ answer $8^{10} y^{40}$
\newline 
problem $\left(7^{2} z^{2}\right)^{9}$ answer $7^{18} z^{18}$
\newline 


## Square Roots
### section 1
#### instruction : Find each square root. (Simplify)

-  easy
  - case 1
  - $ \sqrt{ randomnumber(0, 200) }$
  - problem : $ \sqrt{144} $
  - answer: $ 12 $

-  medium
  - case 1
  - $ \sqrt{ randomnumber(1, 100) }$
  - problem : $ \sqrt{32} $
  - answer: $ 4\sqrt{2} $

-  hard
  - case 1
  - $ \sqrt{ randomnumber(100, 200) }$
  - problem : $ \sqrt{1215} $
  - answer: $ 9\sqrt{15} $


Util

In [ ]:
import math
import random
import sympy


def perfecto_square(n):
   n = int(n)
   sq_root = int(math.sqrt(abs(n)))
   return (sq_root*sq_root) == n

def prime_check(n):
    if n == 1:
        return False
        # from 1 to sqrt(n) 
    for x in range(2, (int)(math.sqrt(n))+1):
        if n % x == 0:
            return False 
    return True

def Square_Roots_Section_1(option_difficulty = "easy", expr= "latex"):
  problemset = ""
  answerset = ""
  negative = ['-', '']
  if option_difficulty == "easy":
    problemset = str(random.choice(negative)) + str(int(random.randint(1, 200)))
    while perfecto_square(problemset) == False:
      problemset = str(random.choice(negative)) + str(int(random.randint(1, 200)))

    if random.randint(1,2) == 1:
      problemset = str(problemset)
      answerset = sympy.sqrt(sympy.simplify(str(problemset)))
      problemset = r"- \sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(sympy.simplify('-'+str(answerset))))
    else:
      answerset = sympy.simplify(sympy.sqrt(sympy.simplify(problemset)))
      problemset = r"\sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(answerset))
    
  elif option_difficulty == "medium":
    problemset = int(random.randint(1, 100))
    while prime_check(problemset) == True:
      problemset = int(random.randint(1, 100))
    if random.randint(1,2) == 1:
      problemset = str(problemset)
      answerset = sympy.sqrt(sympy.simplify(str(problemset)))
      problemset = r"- \sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(sympy.simplify('-'+str(answerset))))
    else:
      answerset = sympy.simplify(sympy.sqrt(sympy.simplify(problemset)))
      problemset = r"\sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(answerset))
  
  elif option_difficulty == "hard":
    problemset = int(random.randint(100, 200))
    while prime_check(problemset) == True:
      problemset = int(random.randint(100, 200))
    if random.randint(1,2) == 1:
      problemset = str(problemset)
      answerset = sympy.sqrt(sympy.simplify(str(problemset)))
      problemset = r"- \sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(sympy.simplify('-'+str(answerset))))
    else:
      answerset = sympy.simplify(sympy.sqrt(sympy.simplify(problemset)))
      problemset = r"\sqrt{" + str(problemset) + "}"
      return(problemset, sympy.latex(answerset))

  #return(problemset, answerset)
problem, answer = Square_Roots_Section_1("easy", "latex")
print(problem, answer) 

problem, answer = Square_Roots_Section_1("medium", "latex")
print(problem, answer) 

problem, answer = Square_Roots_Section_1("hard", "latex")
print(problem, answer) 


\sqrt{81} 9
- \sqrt{88} - 2 \sqrt{22}
\sqrt{160} 4 \sqrt{10}
